# Sentiment-140 Classification

In this notebook, we will explore the application of transformers for sentiment analysis of different tweets. Our goal is to classify tweets to have either positive or negative sentiment. We will first consider the performance of a pre-trained BERT model, finetuned on the Sentiment-140 dataset, and later we'll develop and train our own transformer architecture from scratch. Let's dive right in!

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, BertForSequenceClassification
from data_utils.SentimentDataset import SentimentDataset

# this automatically reloads the libraries so you can update them dynamically
%load_ext autoreload
%autoreload 2

## Loading our data

Let's start by preparing our dataset. We have implemented the dataset class for you in `data_utils/SentimentDataset.py`, feel free to check the implementation. However, before we can use the `SentimentDataset` class to create our train data and test data objects, we need to pre-process the raw data. 

You can download raw data from [here](https://www.kaggle.com/datasets/kazanova/sentiment140). If you examine the raw data file, you can see that there is a lot of redundant information such as the time of each tweet or usernames. For our sentiment analysis, we simply need the tweets and ground truth labels. To preprocess the data file, simply paste the original CSV file in the root project directory without changing the file name and run the script `preprocess_dataset.py`. This should create a new CSV file called `dataset.csv`. (The reason why we do not include the data files on the GitHub repository is that they are simply too big.) Then, you're ready to create a dataset and dataloader below.

In [ ]:
train_data = SentimentDataset('dataset.csv', training_set=True)
test_data = SentimentDataset('dataset.csv', training_set=False)

# train_subset_indices = np.random.choice(len(train_data), size=1000, replace=False)
# train_sampler = SubsetRandomSampler(train_subset_indices)

# test_subset_indices = np.random.choice(len(test_data), size=100, replace=False)
# test_sampler = SubsetRandomSampler(test_subset_indices)

# train_loader = DataLoader(train_data, batch_size=32, sampler=train_sampler)
# test_loader = DataLoader(test_data, batch_size=32, sampler=test_sampler)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

Let's examine a sample tweet from the dataset and print its corresponding ground truth label. Label 1 corresponds to positive sentiment, while label 0 stands for negative sentiment. You can run the cell below multiple times to see different tweets.

In [ ]:
tweets, labels = next(iter(train_loader))
print("Sample tweet: ", tweets[0])
print("Ground truth label: ", labels[0].numpy())

## Fine-tuning BERT

First, let's examine the performance of a pre-trained BERT model on the sentiment analysis task. BERT (original paper [here](https://arxiv.org/abs/1810.04805)) is a language representation model released by Google in 2018 that uses a Bidirectional Transformer architecture (shown below). 

![BERT](BERT.png)

It can be easily fine-tuned on downstream tasks such as sentiment analysis by appending a classification layer to the original BERT architecture. This is already done automatically by importing the `BertForSequenceClassification` from the Hugging Face library called `transformers`. The newly added classification layer is untrained (as you'll see in the warning that appears when you run the following cell) and so we have to fine-tune on the Sentiment-140 data.

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Below, we define a `preprocess` function that takes in the tweets and tokenizes them using the WordPiece algorithm, so that they can be process by our model.

In [ ]:
def preprocess(tweets, tknizer):
    # Tokenize the input batch of texts
    inputs = tknizer(tweets, padding=True, truncation=True, return_tensors="pt")
    return inputs

Next, we need to define a training and evaluation loop.

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, device,
          num_epochs):

    # Place model on device
    model = model.to(device)

    loss_history = []
    acc_history = []

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader),
                  desc=f'Epoch {epoch + 1}/{num_epochs}',
                  position=0,
                  leave=True) as pbar:
            
            for inputs, labels in train_loader:

                inputs = preprocess(inputs, tokenizer)
                # Move inputs and labels to device
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero out gradients
                optimizer.zero_grad()

                # Compute the logits and loss
                outputs = model(**inputs)
                loss = criterion(outputs.logits, labels)

                # Backpropagate the loss
                loss.backward()

                # Update the weights
                optimizer.step()

                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())

                loss_history.append(loss.item())

                    
        avg_loss, accuracy = evaluate(model, val_loader, criterion, device)
        print(
            f'Test set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}'
        )
        acc_history.append(accuracy)

    return loss_history, acc_history


def evaluate(model, test_loader, criterion, device):

    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        total_loss = 0.0
        num_correct = 0
        num_samples = 0

        for inputs, labels in test_loader:

            inputs = preprocess(inputs, tokenizer)
            # Move inputs and labels to device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Compute the logits and loss
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            total_loss += loss.item()

            # Compute the accuracy
            _, predictions = torch.max(outputs.logits, dim=1)
            num_correct += (predictions == labels).sum().item()
            num_samples += len(labels)

    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    accuracy = num_correct / num_samples

    return avg_loss, accuracy

Make sure you are running the model on CUDA.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

We will finetune all the weights in the BERT model (including the pre-trained weights and the newly initialized classifier weights). An alternative approach would be to freeze the pre-trained weights and only fine-tune the classifier weights. You can do that by passing in `model.classifier.parameters()` to the optimizer instead of passing in all the weights of the model.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

loss_history, acc_history = train(model, train_loader, test_loader, optimizer, criterion, device, num_epochs=3)

Next, let's plot our training loss and test accuracy history that documents the fine-tuning process.

In [ ]:
plt.plot(loss_history)
plt.xlabel('Iterations')
plt.ylabel('Cross-entropy Loss')
plt.title('BERT Fine-tuning, Loss history')
plt.show()

In [ ]:
plt.plot(acc_history)
plt.xlabel('Epochs')
plt.ylabel('Test accuracy')
plt.title('BERT Fine-tuning, Accuracy history')
plt.show()

## Creating our own Transformer architecture from scratch

That was fun, right? But maybe slightly too easy, don't you think? Let's try to implement our own transformer architecture now, without the help of pre-trained models. We will base our architecture off of the original paper that introduced transformers, [*Attention is All You Need*](https://arxiv.org/abs/1706.03762).